# Multiclass classification

In [1]:
import pennylane as qml
from pennylane import numpy as np
import torch
from torch.autograd import Variable
import torch.optim as optim

In [2]:
np.random.seed(0)
torch.manual_seed(0)

num_qubits = 2
num_classes = 3
dev = qml.device("default.qubit", wires = 2)

### 1. Data loading and processing

In [3]:
from sklearn import datasets
from sklearn.decomposition import PCA

def load_and_process_data():
    iris = datasets.load_iris()
    X = iris.data
    Y = iris.target
    Y1 = np.zeros((len(Y), 2))
    
    for i in range(len(Y)):
        if Y[i] == 0:
            Y1[i] = [0,0]
        elif Y[i] == 1:
            Y1[i] = [0,1]
        elif Y[i] == 2:
            Y1[i] = [1,0]
        elif Y[i] == 3:
            Y1[i] = [1,1]
    
    return X, Y1

### 2. Quantum Circuit

In [4]:
def layer(W):
    for i in range(num_qubits):
        qml.Rot(*W[i], wires = i)
    qml.CNOT(wires = [0, 1])
    qml.CNOT(wires = [1, 0])

In [28]:
from pennylane.templates import AmplitudeEmbedding

@qml.qnode(dev)
def quantum_circuit(weights, x):
    AmplitudeEmbedding(x, wires = [0,1], normalize = True)  
    for w in weights:
        layer(w)
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))

def Variational_classifier(params, x):
    weights = params[0]
    bias1 = params[1]
    bias2 = params[2]
    
    M1, M2 = quantum_circuit(weights, x)
    return [M1 + bias1, M2 + bias2]

In [40]:
def cost_fn(params, X, labels):
    num_labels = len(labels)
    loss = 0
    predictions = [Variational_classifier(params, x) for x in X]
    for l,p in zip(labels, predictions):
        l1, l2, p1, p2 = l[0], l[1], p[0], p[1]
        loss += (p1 - l1)**2 + (p2 - l2)**2
    loss / num_labels
    return loss

In [30]:
def accuracy(labels, predictions):
    acc = 0
    for l,p in zip(labels, hard_predictions):
        if np.abs(l - p) < 1e-2:
            acc = acc + 1
    acc = acc / len(labels)
    return acc

In [37]:
np.random.seed(0)
num_qubits = 2
num_layers = 2
batch_size = 10
lr_adam = 10
total_iterations = 200

In [41]:
def training(num_layers, steps):
    
    weights = 0.01 * np.random.randn(num_layers, num_qubits, 3)
    bias1 = 0.0
    bias2 = 0.0
    params = [weights, bias1, bias2]
    
    opt = qml.NesterovMomentumOptimizer(0.1)
    
    for i in range(total_iterations):
        batch_index = np.random.choice(len(X), size = batch_size)
        
        X_batch = X[batch_index]
        Y_batch = Y[batch_index]
        params, cost_new = opt.step_and_cost(lambda p: cost_fn(params, X_batch, Y_batch), params)  
        
        if i % 10 == 0:
            print("iteration: ", i, " cost: ", cost_new)
        
    return params

In [42]:
X, Y = load_and_process_data()
params = training(num_layers, total_iterations)

iteration:  0  cost:  7.378325005769595
iteration:  10  cost:  7.064260529991362
iteration:  20  cost:  7.820376627093106
iteration:  30  cost:  6.993499853086551
iteration:  40  cost:  7.976598353406987
iteration:  50  cost:  5.823325578301573
iteration:  60  cost:  8.067993506753579
iteration:  70  cost:  5.859802134403784
iteration:  80  cost:  7.841334205440442
iteration:  90  cost:  7.410295670052437
iteration:  100  cost:  7.810780215926297
iteration:  110  cost:  7.723779875575849
iteration:  120  cost:  7.4288493469254195
iteration:  130  cost:  7.332382656733779
iteration:  140  cost:  7.4062575688328405
iteration:  150  cost:  7.108622663849242
iteration:  160  cost:  7.528895713217365
iteration:  170  cost:  6.821105224052958
iteration:  180  cost:  7.70218236712849
iteration:  190  cost:  7.361691082625536


In [ ]:
bias = torch.tensor([0,0])
bias[0]

In [ ]:
def func(x):
    return torch.tensor([x, x + 1])
X_torch = torch.tensor([1,2,3,4])
X_stack = torch.stack([func(x) for x in X_torch])
print(X_stack)